In [1]:
import fasttext
from sklearn import metrics

In [2]:
model = fasttext.load_model("fasttext-malaya.ftz")

In [3]:
model.predict(['na'], k = 6)[0][0]

['__label__malay', '__label__other']

In [4]:
import json

with open('train-test.json') as fopen:
    train_test = json.load(fopen)
    
train_test.keys()

dict_keys(['train_X', 'test_X', 'train_Y', 'test_Y'])

In [5]:
test_X, test_Y = train_test['test_X'], train_test['test_Y']

In [6]:
model.predict(test_X[0])[0][0].replace('__label__', ''), test_Y[0]

('malay', 'malay')

In [7]:
results = model.predict(test_X[:2])[0]
[r[0].replace('__label__', '') for r in results], test_Y[:2]

(['malay', 'ind'], ['malay', 'ind'])

In [8]:
results[0]

['__label__malay']

In [9]:
from tqdm import tqdm

batch_size = 128
predicted_Y, actual_Y = [], []

for i in tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i: index]
    batch_x = [s.replace('\n','').replace('\t', '') for s in batch_x]
    results = model.predict(batch_x)[0]
    predicted_Y.extend([r[0].replace('__label__', '') for r in results])
    actual_Y.extend(test_Y[i: index])

100%|██████████| 17760/17760 [00:19<00:00, 893.57it/s]


In [10]:
len(predicted_Y) == len(actual_Y)

True

In [11]:
print(
    metrics.classification_report(
        actual_Y,
        predicted_Y,
        digits = 5
    )
)

              precision    recall  f1-score   support

         eng    0.97885   0.90067   0.93814    110129
         ind    0.99524   0.97450   0.98476    354327
       malay    0.95929   0.99686   0.97771   1405848
    manglish    0.99082   0.92965   0.95926    182420
       other    0.96867   0.71465   0.82249     78259
       rojak    0.97184   0.92336   0.94698    142231

    accuracy                        0.96901   2273214
   macro avg    0.97745   0.90662   0.93822   2273214
weighted avg    0.96948   0.96901   0.96815   2273214

